## Updates
- EDM4HEP Schema works with the newstyle FCC samples
- Need some work to ascertain that the relations work (OneToMany Relations have problem)

In [1]:
import awkward as ak

In [2]:
test_file = "../coffea-fcc-analyses/data/edm4hep1/p8_ee_WW_ecm240_edm4hep.root"
from coffea.nanoevents import NanoEventsFactory, FCC
events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass=FCC.get_schema("edm4hep1"),
    delayed = True,
    uproot_options={"filter_name": lambda x: "PARAMETERS" not in x}
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/edm4hep.py:7: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/edm4hep.py:373: UserWarning: No matched collection for edm4hep::Vertex found!
 skipping ...
  warnings.warn(f'No matched collection for {target_datatype} found!\n skipping ...')


In [3]:
events.fields

['CalorimeterHits',
 'EFlowNeutralHadron',
 'EFlowPhoton',
 'EFlowTrack',
 'EFlowTrack_L',
 'EFlowTrack_dNdx',
 'Electron_IsolationVar',
 'Electron_objIdx',
 'EventHeader',
 'GPDoubleKeys',
 'GPDoubleValues',
 'GPFloatKeys',
 'GPFloatValues',
 'GPIntKeys',
 'GPIntValues',
 'GPStringKeys',
 'GPStringValues',
 'Jet',
 'MCRecoAssociations',
 'Muon_IsolationVar',
 'Muon_objIdx',
 'Particle',
 'ParticleIDs',
 'Photon_IsolationVar',
 'Photon_objIdx',
 'ReconstructedParticles',
 'TrackerHits',
 'magFieldBz']

### List Relations
- All collections inherit from the `edm4hep_collection` class
- `edm4hep_collection` class inherits from `base.NanoCollection` class
- Added the function `List_Relations` to list all the available relations(`OneToOne` or `OneToMany`)

In [4]:
events.ReconstructedParticles.List_Relations

{'clusters_idx_EFlowNeutralHadron',
 'clusters_idx_EFlowPhoton',
 'particles_idx_Jet',
 'particles_idx_ReconstructedParticles',
 'tracks_idx_EFlowTrack'}

In [5]:
events.Particle.List_Relations

{'daughters_idx_Particle', 'parents_idx_Particle'}

In [6]:
events.Particle.daughters_idx_Particle.fields

['index', 'collectionID', 'index_Global']

In [7]:
# Note: Seems like the global indices `index_Global` are computed with incorrect values
# Working to fix that.

### Map Relations
- The purpose to create another collection class `edm4hep_collection` is to add a function to map relations, once, so that I dont need to add that over and over again for each datatype class

#### Lets relate the ParticleID datatype with the ReconstructedParticle datatype

- From the edm4hep scheme, we see that, there is a OneToOne relation from `ParticleID` to `ReconstructedParticle`
- In the FCC root files
- `ParticleID` is named as `ParticleIDs`
- `ReconstructedParticle` is named as `ReconstructedParticles`

In [8]:
events.ParticleIDs.List_Relations

{'particle_idx_Jet', 'particle_idx_ReconstructedParticles'}

In [9]:
events.ParticleIDs.Map_Relation('particle', 'ReconstructedParticles').compute()
# Here str 'particle' and 'ReconstructedParticle' are separated only by '_idx_'
# and these two strings are the arguments to Map_Relation
# 'particle' being the generic_name of the relation (as defined in edm4hep.yaml)
# and 'ReconstructedParticles' being the name of the target collection

<ReconstructedParticleArray [[ReconstructedParticle, ...], ...] type='1000 ...'>

In [ ]:
# please note that there is some problem with using Map_Relations to OneToMany Relations,
# Looks like, the relationship branch for OneToMany Relations have incorrect global indices
# Working to fix that